# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f87209d9b20>
├── 'a' --> tensor([[ 0.1419,  0.4819, -0.3666],
│                   [ 0.7945,  1.1199, -0.6234]])
└── 'x' --> <FastTreeValue 0x7f87209d97c0>
    └── 'c' --> tensor([[-0.6200,  0.9693, -1.9223, -0.2201],
                        [-0.2874,  0.0637, -1.5766,  0.8183],
                        [ 1.8783, -0.5559, -0.0795, -0.3217]])

In [4]:
t.a

tensor([[ 0.1419,  0.4819, -0.3666],
        [ 0.7945,  1.1199, -0.6234]])

In [5]:
%timeit t.a

63.9 ns ± 0.0388 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f87209d9b20>
├── 'a' --> tensor([[ 0.1757,  0.3606, -1.5256],
│                   [-0.1950,  0.5269,  0.6023]])
└── 'x' --> <FastTreeValue 0x7f87209d97c0>
    └── 'c' --> tensor([[-0.6200,  0.9693, -1.9223, -0.2201],
                        [-0.2874,  0.0637, -1.5766,  0.8183],
                        [ 1.8783, -0.5559, -0.0795, -0.3217]])

In [7]:
%timeit t.a = new_value

66.5 ns ± 0.0734 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1419,  0.4819, -0.3666],
               [ 0.7945,  1.1199, -0.6234]]),
    x: Batch(
           c: tensor([[-0.6200,  0.9693, -1.9223, -0.2201],
                      [-0.2874,  0.0637, -1.5766,  0.8183],
                      [ 1.8783, -0.5559, -0.0795, -0.3217]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1419,  0.4819, -0.3666],
        [ 0.7945,  1.1199, -0.6234]])

In [11]:
%timeit b.a

59.1 ns ± 0.0227 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.2322,  0.2796, -1.3125],
               [-0.0539, -1.2644, -0.9046]]),
    x: Batch(
           c: tensor([[-0.6200,  0.9693, -1.9223, -0.2201],
                      [-0.2874,  0.0637, -1.5766,  0.8183],
                      [ 1.8783, -0.5559, -0.0795, -0.3217]]),
       ),
)

In [13]:
%timeit b.a = new_value

507 ns ± 0.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

815 ns ± 4.97 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 32.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

140 µs ± 286 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 473 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f86f7edea30>
├── 'a' --> tensor([[[ 0.1419,  0.4819, -0.3666],
│                    [ 0.7945,  1.1199, -0.6234]],
│           
│                   [[ 0.1419,  0.4819, -0.3666],
│                    [ 0.7945,  1.1199, -0.6234]],
│           
│                   [[ 0.1419,  0.4819, -0.3666],
│                    [ 0.7945,  1.1199, -0.6234]],
│           
│                   [[ 0.1419,  0.4819, -0.3666],
│                    [ 0.7945,  1.1199, -0.6234]],
│           
│                   [[ 0.1419,  0.4819, -0.3666],
│                    [ 0.7945,  1.1199, -0.6234]],
│           
│                   [[ 0.1419,  0.4819, -0.3666],
│                    [ 0.7945,  1.1199, -0.6234]],
│           
│                   [[ 0.1419,  0.4819, -0.3666],
│                    [ 0.7945,  1.1199, -0.6234]],
│           
│                   [[ 0.1419,  0.4819, -0.3666],
│                    [ 0.7945,  1.1199, -0.6234]]])
└── 'x' --> <FastTreeValue 0x7f86f7ede3d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.6 µs ± 40.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f867368bb20>
├── 'a' --> tensor([[ 0.1419,  0.4819, -0.3666],
│                   [ 0.7945,  1.1199, -0.6234],
│                   [ 0.1419,  0.4819, -0.3666],
│                   [ 0.7945,  1.1199, -0.6234],
│                   [ 0.1419,  0.4819, -0.3666],
│                   [ 0.7945,  1.1199, -0.6234],
│                   [ 0.1419,  0.4819, -0.3666],
│                   [ 0.7945,  1.1199, -0.6234],
│                   [ 0.1419,  0.4819, -0.3666],
│                   [ 0.7945,  1.1199, -0.6234],
│                   [ 0.1419,  0.4819, -0.3666],
│                   [ 0.7945,  1.1199, -0.6234],
│                   [ 0.1419,  0.4819, -0.3666],
│                   [ 0.7945,  1.1199, -0.6234],
│                   [ 0.1419,  0.4819, -0.3666],
│                   [ 0.7945,  1.1199, -0.6234]])
└── 'x' --> <FastTreeValue 0x7f867368b5e0>
    └── 'c' --> tensor([[-0.6200,  0.9693, -1.9223, -0.2201],
                        [-0.2874,  0.0637, -1.5766,  0.8183],
                 

In [23]:
%timeit t_cat(trees)

29.6 µs ± 51.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.9 µs ± 62.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.1419,  0.4819, -0.3666],
                [ 0.7945,  1.1199, -0.6234]],
       
               [[ 0.1419,  0.4819, -0.3666],
                [ 0.7945,  1.1199, -0.6234]],
       
               [[ 0.1419,  0.4819, -0.3666],
                [ 0.7945,  1.1199, -0.6234]],
       
               [[ 0.1419,  0.4819, -0.3666],
                [ 0.7945,  1.1199, -0.6234]],
       
               [[ 0.1419,  0.4819, -0.3666],
                [ 0.7945,  1.1199, -0.6234]],
       
               [[ 0.1419,  0.4819, -0.3666],
                [ 0.7945,  1.1199, -0.6234]],
       
               [[ 0.1419,  0.4819, -0.3666],
                [ 0.7945,  1.1199, -0.6234]],
       
               [[ 0.1419,  0.4819, -0.3666],
                [ 0.7945,  1.1199, -0.6234]]]),
    x: Batch(
           c: tensor([[[-0.6200,  0.9693, -1.9223, -0.2201],
                       [-0.2874,  0.0637, -1.5766,  0.8183],
                       [ 1.8783, -0.5559, -0.0795, -0.3217]],
         

In [26]:
%timeit Batch.stack(batches)

77.4 µs ± 318 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.1419,  0.4819, -0.3666],
               [ 0.7945,  1.1199, -0.6234],
               [ 0.1419,  0.4819, -0.3666],
               [ 0.7945,  1.1199, -0.6234],
               [ 0.1419,  0.4819, -0.3666],
               [ 0.7945,  1.1199, -0.6234],
               [ 0.1419,  0.4819, -0.3666],
               [ 0.7945,  1.1199, -0.6234],
               [ 0.1419,  0.4819, -0.3666],
               [ 0.7945,  1.1199, -0.6234],
               [ 0.1419,  0.4819, -0.3666],
               [ 0.7945,  1.1199, -0.6234],
               [ 0.1419,  0.4819, -0.3666],
               [ 0.7945,  1.1199, -0.6234],
               [ 0.1419,  0.4819, -0.3666],
               [ 0.7945,  1.1199, -0.6234]]),
    x: Batch(
           c: tensor([[-0.6200,  0.9693, -1.9223, -0.2201],
                      [-0.2874,  0.0637, -1.5766,  0.8183],
                      [ 1.8783, -0.5559, -0.0795, -0.3217],
                      [-0.6200,  0.9693, -1.9223, -0.2201],
                      [-0.2874,  

In [28]:
%timeit Batch.cat(batches)

144 µs ± 462 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

324 µs ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
